In [1]:
import sys, os 

import numpy as np
import pandas as pd

import scipy as sci
import xarray as xr

# Constants

In [2]:
""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

# Setup

In [3]:
input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
input_files['thetao']

<xarray.DataArray 'thetao' (time: 60, depth_coord: 21, latitude: 10, longitude: 22)>
dask.array<concatenate, shape=(60, 21, 10, 22), dtype=float32, chunksize=(12, 21, 10, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
  * longitude    (longitude) float64 -11.75 -11.25 -10.75 ... -2.25 -1.75 -1.25
  * latitude     (latitude) float64 47.25 47.75 48.25 ... 50.75 51.25 51.75
  * depth_coord  (depth_coord) float32 0.0 10.0 20.0 30.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [4]:
input_files['thetao'].transpose('time','latitude','longitude','depth_coord')[0][0][13][0].load()

<xarray.DataArray 'thetao' ()>
array(12.1378145, dtype=float32)
Coordinates:
    time         datetime64[ns] 2000-01-31
    longitude    float64 -5.25
    latitude     float64 47.25
    depth_coord  float32 0.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [5]:
n=273
m = input_files['thetao'].transpose('latitude','longitude', 'time', 'depth_coord').fillna(-999); m
def doit():
    i = 0
    for lat in range(len(m.latitude)):
        for lon in range(len(m.longitude)):
            for d in range(len(m.depth_coord)):
                if i == n:
                    print(lat,lon,d)
                    return
                i+=1
doit()

0 13 0


In [33]:
m[0][13][0][0].values.ravel()[0]

12.1378145

In [51]:
def saveIt():
    m = input_files['thetao'].transpose('latitude','longitude', 'time', 'depth_coord').fillna(-999)
    len_lat, len_lon = len(m.latitude), len(m.longitude)
    len_ts, len_depth = len(m.time), len(m.depth_coord)

    output = np.arange(len_lat*len_lon*len_depth * len_ts).reshape(len_lat, len_lon, len_depth, len_ts).astype('float32')
    m = input_files['thetao'].transpose('latitude','longitude', 'depth_coord', 'time').fillna(-999).astype('float32')

    for lat in range(len_lat):
        for lon in range(len_lon):
            for depth in range(len_depth):
                output[lat,lon,depth] = m[lat,lon,depth,:].load()

    output = output.reshape(len_lat * len_lon * len_depth, len_ts)  
    # mask = output < -999; mask
    # output[mask] = -999
    np.savetxt('data_2000-2004.csv', output.astype('float32'), delimiter=',', fmt='%.6f')
saveIt()

In [52]:
# lat=05, lon=5,depth=5
m[5,5,5].load()

<xarray.DataArray 'thetao' (time: 60)>
array([11.401852 , 10.927594 , 10.693255 , 10.832945 , 11.089084 ,
       11.140119 , 11.144329 , 11.018477 , 11.917805 , 12.182155 ,
       11.372435 , 11.440055 , 11.309369 , 11.111829 , 10.965912 ,
       10.83427  , 10.905318 , 11.655176 , 13.288745 , 14.76556  ,
       14.653581 , 14.136719 , 13.54315  , 12.99344  , 11.943633 ,
       11.663304 , 11.552512 , 11.454192 , 11.846073 , 12.198486 ,
       12.230988 , 12.585258 , 13.073701 , 13.285204 , 13.414214 ,
       13.196177 , 12.406819 , 11.4618025, 11.405766 , 11.485776 ,
       12.20816  , 12.470518 , 12.60076  , 12.989866 , 13.011666 ,
       12.836416 , 13.041643 , 12.566862 , 12.053412 , 11.171542 ,
       10.90778  , 10.902459 , 11.245266 , 11.252755 , 11.290906 ,
       11.852666 , 11.937502 , 11.983492 , 12.534066 , 12.223845 ],
      dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
    longitude    float64 -9.25
    latitude     float64 49.75
    depth_coord  float32 50.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [53]:
n_lat=10
n_lon=22
n_depths=21
n_time=60
result = np.empty((n_time,n_lat,n_lon,n_depths))

lat,lon = 0,0
for i, row in enumerate(np.loadtxt('data_2000-2004.csv', delimiter=',')):
    d = i % n_depths
    if i % n_depths == 0 and i != 0: lon+=1
    if lon == n_lon:
        lon=0
        lat+=1
    if lat == n_lat: break
    for time in range(n_time): 
        result[time,lat,lon,d] = row[time]    
result[:,5,5,5]

array([11.401852, 10.927594, 10.693255, 10.832945, 11.089084, 11.140119,
       11.144329, 11.018477, 11.917805, 12.182155, 11.372435, 11.440055,
       11.309369, 11.111829, 10.965912, 10.83427 , 10.905318, 11.655176,
       13.288745, 14.76556 , 14.653581, 14.136719, 13.54315 , 12.99344 ,
       11.943633, 11.663304, 11.552512, 11.454192, 11.846073, 12.198486,
       12.230988, 12.585258, 13.073701, 13.285204, 13.414214, 13.196177,
       12.406819, 11.461802, 11.405766, 11.485776, 12.20816 , 12.470518,
       12.60076 , 12.989866, 13.011666, 12.836416, 13.041643, 12.566862,
       12.053412, 11.171542, 10.90778 , 10.902459, 11.245266, 11.252755,
       11.290906, 11.852666, 11.937502, 11.983492, 12.534066, 12.223845])

# Compute

In [55]:
""" Definitions of kinetic functions """
Kelvin = 273.15       
def equation2(input_temp):
    temperature_kelvin = input_temp + Kelvin 
    # print(temperature_kelvin[-10:])
    # Calculate a
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    # print(f'Anumeratr:{a_numerator}')
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    return a_numerator/a_denominator

def equation3(input_temp):
    """ Arrhenius equation """
    temperature_kelvin = input_temp + Kelvin
    # Calculate b
    return B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)



experiment_name = 'SODA'
region = 'CelticSea'

# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 2 
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 2000    
# Number of years in the input temperature dataset
years = range(2000, 2005)  	 


# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
lat_coords = slice(47,52)
lon_coords = slice(-12,-1)

# Define depth_levels for your growth model output files
depth_levels = slice(0, 600)   # 0-600 meters according to Atlantic cod distribution

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array([47.25, 47.75, 48.25, 48.75, 49.25, 49.75, 50.25, 50.75, 51.25, 51.75], dtype='f')
lon = np.array([
     -11.75, -11.25, -10.75, -10.25, -9.75, -9.25, -8.75, -8.25, -7.75,
    -7.25, -6.75, -6.25, -5.75, -5.25, -4.75, -4.25, -3.75, -3.25, -2.75, -2.25, -1.75,
    -1.25
], dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array([
     -0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 115.,
     135., 160., 190., 230., 280., 340., 410., 490., 580.
], dtype='f')

dt = 1

# Define dimensionality of coords
N_depths = len(depths)
N_lat = len(lat)
N_lon = len(lon)

input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})

# Identifiyng the last year (death of cod)
last_year = initial_year + generation    
while last_year <= years[-1]:
    print('------------------------------------')
    age = 0                                       
    
    weight = np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(
            time=f'{every_year}', 
            depth_coord = depth_levels, latitude = lat_coords, longitude = lon_coords
        )        

        temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             

        # Set NaN values                         
        temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan        
        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
        b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

        if every_year == initial_year: mm0 = 2           
        else: mm0 = 0
        # print(temp_input_3d[1,0,13:19])
        for mon in np.arange(0,12):
            for dd in np.arange(0,30):
                for ilev in np.arange(0, N_depths):  
                    a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                    b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                    print(a[ilev,-20:])
                    break
                break
            break


                    # growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  
                    # growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0, 0, growth_rates[ilev, :])
                    # weight[ilev, :] = weight[ilev, :] * (1. + dt  * growth_rates[ilev, :])
                    # if ilev == 0 and mon == 7 and dd == 0:
                    #     print(a[ilev,18], weight[ilev,18], b[ilev,18], C_AVG)
                    
                    # for i,j in enumerate(growth_rates[ilev]):
                    #     if j < 0:
                    #         print(f'{i} - {j} ilev:{ilev}, mon: {mon}, dd: {dd}')
                    #         print(growth_rates[ilev][18])





        # print(weight[0,13:19])


        #             if ilev == 20:
        #                 print(weight[ilev,-10:])
        #                 break
        #         break
        #     break
            # print(f'{mon}',weight[0,13:19]) 
        # break
        # print(weight.shape)
        # print(weight[0,10:20]) 
         
        
                                                            
        new_year = int(every_year) + 1
        
        # Reshape data to original shape
        a_3d = a.reshape(N_depths, N_lat, N_lon)
        b_3d = b.reshape(N_depths, N_lat, N_lon)
        growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)
        
        # 3D field with asymptotic weight
        weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
        
        # Calculate maximum asymptotic weight at a given location 
        # ("W*" in Butzin and Pörtner (2016)) 
        weight_max = np.nanmax(weight_3d, axis = 0)
      

    # Identifiyng the last year (death of cod)
    last_year = initial_year + generation    
    initial_year += 1

------------------------------------
[7.0714526 6.478852  6.0981526 6.527761  6.602787  6.6033735 6.6580367
 6.7115026 6.809083  6.823108  6.6745014 6.3167644 5.6622224 6.1244364
 6.193604        nan       nan       nan       nan       nan]
[6.3037314 5.815156  5.5441422 5.6291842 5.648591  5.7096977 5.836475
 5.8928533 5.864888  5.8578243 5.8513584 5.477367  4.952949  5.3258553
 5.3114667       nan       nan       nan       nan       nan]
------------------------------------
[6.3037314 5.815156  5.5441422 5.6291842 5.648591  5.7096977 5.836475
 5.8928533 5.864888  5.8578243 5.8513584 5.477367  4.952949  5.3258553
 5.3114667       nan       nan       nan       nan       nan]
------------------------------------
[7.849229  7.284911  6.8417263 6.965671  6.876864  6.8782873 7.0409975
 7.253695  7.4180017 7.3744187 7.139333  6.838722  6.3270226 6.8408155
 6.909857        nan       nan       nan       nan       nan]
------------------------------------
[7.5828767 6.8823566 6.5074425 6.91213

In [15]:
for i,_ in enumerate(range(3)):
    print(i)

0
1
2


In [115]:
arr = np.array([np.nan, -1, 1])

In [116]:
np.where(arr < 0, 0, arr)

array([nan,  0.,  1.])